# Generate and Save Synthetic Data for Manifolds

In [1]:
#JAX
import jax.numpy as jnp
from jax import vmap

#numpy
import numpy as np

#argparse
import argparse

from ManLearn.VAE.VAE_MNIST import model as mnist_model
from ManLearn.VAE.VAE_MNIST import model_encoder as mnist_encoder
from ManLearn.VAE.VAE_MNIST import model_decoder as mnist_decoder
from ManLearn.VAE.VAE_MNIST import VAEOutput as mnist_output

from ManLearn.VAE.VAE_SVHN import model as svhn_model
from ManLearn.VAE.VAE_SVHN import model_encoder as svhn_encoder
from ManLearn.VAE.VAE_SVHN import model_decoder as svhn_decoder
from ManLearn.VAE.VAE_SVHN import VAEOutput as svhn_output

from ManLearn.VAE.VAE_CelebA import model as celeba_model
from ManLearn.VAE.VAE_CelebA import model_encoder as celeba_encoder
from ManLearn.VAE.VAE_CelebA import model_decoder as celeba_decoder
from ManLearn.VAE.VAE_CelebA import VAEOutput as celeba_output

from ManLearn.train_MNIST import load_dataset as load_mnist
from ManLearn.train_SVHN import load_dataset as load_svhn
from ManLearn.train_CelebA import load_dataset as load_celeba
from ManLearn.model_loader import load_model

#jaxgeometry
from jaxgeometry.manifolds import Euclidean, nSphere, Ellipsoid, Cylinder, S1, Torus, \
    H2, Landmarks, Heisenberg, SPDN, Latent, HypParaboloid
from jaxgeometry.setup import dts, dWs, hessianx
from jaxgeometry.statistics import score_matching
from jaxgeometry.statistics.score_matching.model_loader import load_model
from jaxgeometry.stochastics import Brownian_coords, product_sde, Brownian_sR
from jaxgeometry.stochastics.product_sde import tile

2023-10-14 23:25:12.870305: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


## Hyper-Parameters

In [2]:
N_sim = 100
n_steps = 100
T = 5.0

file_path = 'Data/'

## Euclidean

### Generate Data

In [3]:
dim = [2,3,5,10,20,50,100,200]
for d in dim:
    M = Euclidean(N=d)
    Brownian_coords(M)
    
    (product, sde_product, chart_update_product) = product_sde(M, 
                                                               M.sde_Brownian_coords, 
                                                               M.chart_update_Brownian_coords)

    x0 = M.coords([0.]*d)
    x0s = tile(x0, N_sim)
    
    _dts = dts(T=T, n_steps=n_steps)
    dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
    (ts,xss,chartss,*_) = product(x0s,
                                  _dts,dW,jnp.repeat(1.,N_sim))
    
    xs = xss[-1]
    chart = chartss[-1]
    
    path = ''.join((file_path, 'R', str(d), '/'))
    np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
    np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
Done


## S1

### Generate Data

In [23]:
M = S1(use_spherical_coords=True)
Brownian_coords(M)

x0 = M.coords([0.])
x0s = tile(x0, N_sim)

(product, sde_product, chart_update_product) = product_sde(M, 
                                                            M.sde_Brownian_coords, 
                                                            M.chart_update_Brownian_coords)

_dts = dts(T=0.1, n_steps=n_steps)
dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
(ts,xss,chartss,*_) = product(x0s,
                              _dts,dW,jnp.repeat(1.,N_sim))

xs = xss[-1]
chart = chartss[-1]
chart = vmap(lambda x,y: M.F((x,y)))(xs,chart)

path = ''.join((file_path, 'S1/'))
np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
Done


## mSphere

### Generate Data

In [ ]:
dim = [2,3,5,10,20,50,100, 200]
for d in dim:
    M = nSphere(N=d)
    Brownian_coords(M)

    (product, sde_product, chart_update_product) = product_sde(M, 
                                                               M.sde_Brownian_coords, 
                                                               M.chart_update_Brownian_coords)

    x0 = M.coords([0.]*d)
    x0s = tile(x0, N_sim)
    
    _dts = dts(T=T, n_steps=n_steps)
    dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
    (ts,xss,chartss,*_) = product(x0s,
                                  _dts,dW,jnp.repeat(1.,N_sim))
    
    xs = xss[-1]
    chart = chartss[-1]
    
    path = ''.join((file_path, 'S', str(d), '/'))
    np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
    np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm


## Ellipsoid

### Generate Data

In [9]:
## mSphere

### Generate Data

dim = [2,3,5,10,20]
for d in dim:
    M = Ellipsoid(N=d, params = jnp.linspace(0.5,1.0,d+1))
    Brownian_coords(M)

    N_dim = M.emb_dim
    x0 = M.coords([0.]*d)

    (product, sde_product, chart_update_product) = product_sde(M, 
                                                               M.sde_Brownian_coords, 
                                                               M.chart_update_Brownian_coords)

    x0s = tile(x0, N_sim)
    
    _dts = dts(T=T, n_steps=n_steps)
    dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
    (ts,xss,chartss,*_) = product(x0s,
                                  _dts,dW,jnp.repeat(1.,N_sim))
    
    xs = xss[-1]
    chart = chartss[-1]
    
    path = ''.join((file_path, 'Ellipsoid', str(d), '/'))
    np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
    np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
Done


## Cylinder

### Generate Data

In [15]:
## mSphere

### Generate Data

M = Cylinder(params=(1.,jnp.array([0.,0.,1.]),jnp.pi/2.))
Brownian_coords(M)

x0 = M.coords([0.]*2)

(product, sde_product, chart_update_product) = product_sde(M, 
                                                           M.sde_Brownian_coords, 
                                                           M.chart_update_Brownian_coords)

x0s = tile(x0, N_sim)

_dts = dts(T=T, n_steps=n_steps)
dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
(ts,xss,chartss,*_) = product(x0s,
                              _dts,dW,jnp.repeat(1.,N_sim))

xs = xss[-1]
chart = chartss[-1]
path = ''.join((file_path, 'Cylinder/'))
np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
Done


## Torus

### Generate Data

In [16]:
## mSphere

### Generate Data

M = Torus()
Brownian_coords(M)

x0 = M.coords([0.]*2)

(product, sde_product, chart_update_product) = product_sde(M, 
                                                           M.sde_Brownian_coords, 
                                                           M.chart_update_Brownian_coords)

x0s = tile(x0, N_sim)

_dts = dts(T=T, n_steps=n_steps)
dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
(ts,xss,chartss,*_) = product(x0s,
                              _dts,dW,jnp.repeat(1.,N_sim))

xs = xss[-1]
chart = chartss[-1]

path = ''.join((file_path, 'Torus/'))
np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
Done


## Landmarks

### Generate Data

In [7]:
## mSphere

### Generate Data

dim = [2,5,10,20]
for d in dim:
    M = Landmarks(N=d,m=2)
    Brownian_coords(M)

    N_dim = M.dim
    x0 = M.coords(jnp.vstack((jnp.linspace(-5.0,5.0,M.N),jnp.linspace(5.0,-5.0,M.N))).T.flatten())
    x0 = M.coords(jnp.vstack((jnp.linspace(-5.0,5.0,M.N),jnp.zeros(M.N))).T.flatten())

    (product, sde_product, chart_update_product) = product_sde(M, 
                                                               M.sde_Brownian_coords, 
                                                               M.chart_update_Brownian_coords)

    #x0 = M.coords(jnp.vstack((jnp.linspace(-10.0,10.0,M.N),jnp.linspace(10.0,-10.0,M.N))).T.flatten())
    x0 = M.coords(jnp.vstack((jnp.linspace(-5.0,5.0,M.N),jnp.zeros(M.N))).T.flatten())

    if M.N >=10:
        with open('../Data/landmarks/Papilonidae/Papilionidae_landmarks.txt', 'r') as the_file:
            all_data = [line.strip() for line in the_file.readlines()]

            x1 = jnp.array([float(x) for x in all_data[0].split()[2:]])
            x2 = jnp.array([float(x) for x in all_data[1].split()[2:]])

            x0 = M.coords(jnp.vstack((x1[::len(x1)//M.N],x2[::len(x2)//M.N])).T.flatten())
    
    x0s = tile(x0, N_sim)
    _dts = dts(T=T, n_steps=n_steps)
    dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
    (ts,xss,chartss,*_) = product(x0s,
                                  _dts,dW,jnp.repeat(1.,N_sim))
    
    xs = xss[-1]
    chart = chartss[-1]
    
    path = ''.join((file_path, 'Landmarks', str(d), '/'))
    np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
    np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
using M.Exp for Logarithm
Done


## SPDN

### Generate Data

In [3]:
## mSphere

### Generate Data
from jaxgeometry.stochastics.GRW import initialize
dim = [2]#[2, 5, 10]
for d in dim:
    M = SPDN(N=d)    
    Brownian_coords(M)
    initialize(M)

    N_dim = M.emb_dim
    x0 = M.coords([10.]*(d*(d+1)//2))

    (product, sde_product, chart_update_product) = product_sde(M, 
                                                               M.sde_Brownian_coords, 
                                                               M.chart_update_Brownian_coords)

    x0s = tile(x0, N_sim)
    
    _dts = dts(T=T, n_steps=n_steps)
    dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
    #(ts,xss,chartss,*_) = product(x0s,
    #                              _dts,dW,jnp.repeat(1.,N_sim))
    (ts,xss,chartss) = M.product_GRW(x0s,_dts,dW)
    print(xss)
    
    xs = xss[-1]
    chart = chartss[-1]
    
    path = ''.join((file_path, 'SPDN', str(d), '/'))
    np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
    np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
[[[10.253797   9.819727  10.598152 ]
  [ 9.678076  10.01162   10.094679 ]
  [10.2175455 10.07594    9.72322  ]
  ...
  [ 9.948575   9.967027  10.325599 ]
  [ 9.650476  10.0071125  9.994855 ]
  [10.191658  10.119114  10.32426  ]]

 [[10.652054   9.701954  10.763684 ]
  [ 9.760528  10.048314   9.879705 ]
  [ 9.49432    9.782054   9.181025 ]
  ...
  [10.03166   10.125913  10.376295 ]
  [ 9.513203   9.608871   9.9888115]
  [10.140708  10.230803  10.569252 ]]

 [[10.797464   9.943055  10.5380535]
  [ 9.867455  10.11319   10.201972 ]
  [ 9.808816  10.159669   9.148694 ]
  ...
  [ 9.679119   9.954854  10.274313 ]
  [ 9.74958    9.843973   9.923022 ]
  [10.060341  10.283753  10.263904 ]]

 ...

 [[10.87924   11.1395645  9.859633 ]
  [ 5.0518074 16.604252  12.042284 ]
  [10.595396  11.304392  10.064004 ]
  ...
  [11.928803  10.4301195 10.222655 ]
  [10.697912   9.629325  10.67621  ]
  [ 9.7191305 11.767609   8.868145 ]]

 [[11.0544195 11.265483  10.050838 ]
  [ 5.04703

## Hyperbolic Paraboloid

### Generate Data

In [24]:
## mSphere

### Generate Data

M = HypParaboloid()
Brownian_coords(M)

x0 = M.coords([0.]*2)

(product, sde_product, chart_update_product) = product_sde(M, 
                                                           M.sde_Brownian_coords, 
                                                           M.chart_update_Brownian_coords)

x0s = tile(x0, N_sim)

_dts = dts(T=T, n_steps=n_steps)
dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
(ts,xss,chartss,*_) = product(x0s,
                              _dts,dW,jnp.repeat(1.,N_sim))

xs = xss[-1]
chart = chartss[-1]

path = ''.join((file_path, 'HypParaboloid/'))
np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
Done


## VAE MNIST

### Generate Data

In [27]:
## mSphere

### Generate Data

dim = [2, 5, 10]
ds = load_mnist("train", 100, 2712)
        
state = load_model('ManLearn/models/MNIST/VAE/')
F = lambda x: mnist_decoder.apply(state.params, state.rng_key, x[0]).reshape(-1)

M = Latent(F=F,dim=2,emb_dim=28*28,invF=None)
Brownian_coords(M)

x0 = mnist_encoder.apply(state.params, state.rng_key, next(ds).image)
x0 = M.coords(x0[0])
        
(product, sde_product, chart_update_product) = product_sde(M, 
                                                               M.sde_Brownian_coords, 
                                                               M.chart_update_Brownian_coords)

x0s = tile(x0, N_sim)

_dts = dts(T=T, n_steps=n_steps)
dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
(ts,xss,chartss,*_) = product(x0s,
                              _dts,dW,jnp.repeat(1.,N_sim))

xs = xss[-1]
chart = chartss[-1]

path = ''.join((file_path, 'VAE_MNIST/'))
np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

2023-10-11 00:45:40.381165: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


using M.Exp for Logarithm
Done


## VAE SVHN

### Generate Data

In [15]:
## mSphere

### Generate Data

dim = [2, 5, 10]
ds = load_svhn()
        
state = load_model('ManLearn/models/SVHN/VAE/')
F = lambda x: svhn_decoder.apply(state.params, state.rng_key, x[0]).reshape(-1)

M = Latent(F=F,dim=32,emb_dim=32*32*3,invF=None)
Brownian_coords(M)

x0 = svhn_encoder.apply(state.params, state.rng_key, next(ds).image)
x0 = M.coords(x0[0])
        
(product, sde_product, chart_update_product) = product_sde(M, 
                                                               M.sde_Brownian_coords, 
                                                               M.chart_update_Brownian_coords)

x0s = tile(x0, N_sim)

_dts = dts(T=T, n_steps=n_steps)
dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
(ts,xss,chartss,*_) = product(x0s,
                              _dts,dW,jnp.repeat(1.,N_sim))

xs = xss[:,-1]
chart = chartss[:,-1]

path = ''.join((file_path, 'VAE_SVHN/'))
np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

2023-10-02 20:00:02.191492: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


using M.Exp for Logarithm
Done


## VAE CelebA

### Generate Data

In [16]:
## mSphere

### Generate Data

dim = [2, 5, 10]
ds = load_celeba()
        
state = load_model('ManLearn/models/CelebA/VAE/')
F = lambda x: celeba_decoder.apply(state.params, state.rng_key, x[0]).reshape(-1)

M = Latent(F=F,dim=32,emb_dim=64*64*3,invF=None)
Brownian_coords(M)

N_dim = M.dim
x0 = svhn_encoder.apply(state.params, state.rng_key, next(ds).image)
x0 = M.coords(x0[0])

(product, sde_product, chart_update_product) = product_sde(M, 
                                                               M.sde_Brownian_coords, 
                                                               M.chart_update_Brownian_coords)

x0s = tile(x0, N_sim)

_dts = dts(T=T, n_steps=n_steps)
dW = dWs(N_sim*M.dim,_dts).reshape(-1,N_sim,M.dim)
(ts,xss,chartss,*_) = product(x0s,
                              _dts,dW,jnp.repeat(1.,N_sim))

xs = xss[:,-1]
chart = chartss[:,-1]

path = ''.join((file_path, 'VAE_CelebA/'))
np.savetxt(''.join((path, 'xs.csv')), xs, delimiter=",")
np.savetxt(''.join((path, 'chart.csv')), chart, delimiter=",")
print("Done")

using M.Exp for Logarithm
Done
